imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans

In [2]:
# Define output folder path
intermediate_output_folder_path = os.path.join('..', '..', 'data','02_intermediate_output')

# Define data to process folder path
data_folder_path = os.path.join('..','..', 'data','01_input_data')

In [3]:
grids_path= os.path.join(intermediate_output_folder_path, 'crime_index_zonal_statistics.gpkg')

grids = gpd.read_file(grids_path)

In [4]:
grids.head()

,Cumulative_opportunity_measure_to_access_jobs_in_60_minutes,Travel_time_to_closest_public_school,Travel_time_to_closest_healthcare_facility,Number_of_buildings,Number_of_residents_above_60_years,Number_of_residents_between_15_and_24_years_old,Population,Illiteracy_rate,Average_household_income,Number_of_shootings,...,apai_arrest_total,arrest_warrant_total,search_warrant_and_seizure_total,threat_total,missing_people_total,cadaver_found_total,skeleton_found_total,military_police_dead_in_service_total,civil_police_dead_in_service_total,geometry
0,0.0,30.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0,...,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0,"MULTIPOLYGON (((623543.654 7466506.750, 623553..."
1,0.0,30.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0,...,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0,"MULTIPOLYGON (((623542.434 7466324.100, 623540..."
2,0.0,30.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0,...,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0,"MULTIPOLYGON (((623571.614 7466230.200, 623558..."
3,0.0,30.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0,...,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0,"MULTIPOLYGON (((623718.395 7466708.231, 623757..."
4,0.0,30.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0,...,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0,"MULTIPOLYGON (((623572.224 7466605.211, 623581..."


invert these fields 

In [8]:
columns_to_invert = ['Travel_time_to_closest_healthcare_facility', 
                     'Average_household_income', 
                     'OSM_road_network_centraility_by_average_travel_time_in_car', 
                     'Normalized_difference_vegetation_index',
                     "Perceptual_GeoAI_safety_score_from_street_view_imagery",
                     'Coverage_by_Urban_area', 
                     'Number_of_buildings', 
                     'Average_building_height', 
                     'Coverage_of_built_up_area', 
                     'Building_completeness', 
                     'Satellite_nightlight_saturation', 
                     'Number_of_streetlights', 
                     'Number_of_CCTV_camera', 
                     'Coverage_by_pacification_police_area']

                     #    'Number_of_residents_above_60_years', 

# Invert each column while maintaining the distribution
for col in columns_to_invert:
    grids_sorted = grids.sort_values(by=col, ascending=False)
    max_value = grids_sorted[col].max()
    min_value = grids_sorted[col].min()
    grids_sorted[col] = max_value - grids_sorted[col] + min_value
    grids = grids_sorted.sort_index()


## Define  non-CISP and CISP columns

### categorize the df into two categories:
- **'cisp_mean' column:** crime records from police data.
- **'noncisp_mean' column:**  other crime index data from previous analyses.

Each entry is associated with a specific 'CISP_ID'.


In [9]:
# Define the non-CISP and CISP columns
noncisp_col = [
       #Zonal statistics QGIS
       'Cumulative_opportunity_measure_to_access_jobs_in_60_minutes', 
       'Travel_time_to_closest_healthcare_facility', 
       'Number_of_buildings',
       'Illiteracy_rate',  
       'Average_household_income', 
       'Satellite_nightlight_saturation',  
       'Average_building_height', 
       'Normalized_difference_vegetation_index',
       'OSM_road_network_centraility_by_average_travel_time_in_car', 
       'Perceptual_GeoAI_safety_score_from_street_view_imagery' , 
       
       #count points QGIS   
       'Number_of_streetlights', 
       'Number_of_CCTV_camera',  

       ##Zonal statistics Script
       'Distance_to_closest_military_station', 
       'Distance_to_closest_police_station', 
       'Coverage_by_Urban_area', 
       'Coverage_by_pacification_police_area',
       'Coverage_by_Favela_area',
       'Coverage_of_built_up_area', 
       'Building_completeness',
       'Coverage_by_Militia_gang_area',
       'Coverage_by_Terceiro Comando Puro_gang_area',
       'Coverage_by_Amigo_dos_amigos_gang_area',
       'Coverage_by_Comando_Vermelho_gang_area',
   ]

# noncisp exlcuding 
       # - Number_of_shootings  & Coverage_by_Em_disputa_gang_area 'Population',       'Number_of_residents_above_60_years',
       # 'Number_of_residents_between_15_and_24_years_old',         'Travel_time_to_closest_public_school',
 
cisp_col = ['intentional_homicide_total', 'bodily_injury_death_total',
       'robbery_with_death_total', 'violent_crime_total',
       'homicide_by_police_intervention_total', 'violent_lethality_total',
       'attempted_homicide_total', 'intentional_bodily_injury_total',
       'rape_total', 'culpable_homicide_total', 'culpable_bodily_injury_total',
       'mugger_robbery_total', 'mobile_phone_robbery_total',
       'collective_robbery_total', 'street_robbery_total',
       'vehicle_robbery_total', 'cargo_theft_total',
       'commercial_robbery_total', 'residential_robbery_total',
       'bank_robbery_total', 'atm_robbery_total',
       'robbery_during_withdrawal_total', 'robbery_after_withdrawal_total',
       'bicycle_robbery_total', 'other_robberies_total',
       # 'total_robberies_total',
         'vehicle_theft_total',
       'pedestrian_theft_total', 'collective_theft_total',
       'mobile_phone_theft_total', 'bicycle_theft_total', 'other_thefts_total',
       # 'total_thefts_total',
         'kidnapping_total', 'extortion_total',
       'express_kidnapping_total', 'fraud_total', 'drug_seizure_total',
       'drug_possession_total', 'drug_trafficking_total',
       'drug_seizure_without_author_total', 'vehicle_recovery_total',
       'federal_police_arrest_total', 'apai_arrest_total',
       'arrest_warrant_total', 'search_warrant_and_seizure_total',
       'threat_total', 'missing_people_total', 'cadaver_found_total',
       'skeleton_found_total', 'military_police_dead_in_service_total',
       'civil_police_dead_in_service_total']
   


In [10]:
grids['sum_of_cisp_col'] = grids[cisp_col].sum(axis=1)

## standardize the df

In [11]:
# standardize func for cisp and noncisp columns
def standardize_data(df, columns, prefix):
    
    # Create a copy of the DataFrame with only the specified columns
    data = df[columns].copy()

    # Initialize a MinMaxScaler
    standard_scaler = MinMaxScaler()

    # Fit and transform the data
    data_transformed = standard_scaler.fit_transform(data)

    # Convert the transformed data back into a DataFrame
    data_df = pd.DataFrame(data_transformed, columns=[prefix + '_' + col for col in columns])

    # Concatenate the original DataFrame with the standardized values
    df = pd.concat([df, data_df], axis=1)

    # Calculate the mean of the standardized values for each row
    df[prefix + '_mean'] = data_df.mean(axis=1)

    return df


In [12]:

# Standardize non-CISP and CISP columns > 
grids = standardize_data(grids, noncisp_col, 'noncisp')

# grids = standardize_data(grids, noncisp_col, 'noncisp')
grids = standardize_data(grids, cisp_col, 'cisp')

# Concatenate mean columns and 'Cisp' column
crime_df = pd.concat([grids['cisp_mean'], grids['Cisp_ID'], grids['noncisp_mean']], axis=1)

crime_df.head()

,cisp_mean,Cisp_ID,noncisp_mean
0,0.33599,36.0,0.563940
1,0.33599,36.0,0.561908
2,0.33599,36.0,0.562423
3,0.33599,36.0,0.560358
4,0.33599,36.0,0.558823


## donwscaling crime records from police data.

### Redistributing 'cisp_mean' Values Based on 'noncisp_mean' for Each 'CISP_ID' in the DataFrame

In [13]:
def redistribute(df, column_name):
    # Count occurrences of each 'Cisp' value
    cisp_value_counts = df['Cisp_ID'].value_counts()

    # Create a DataFrame with 'Cisp' counts
    count_df = pd.DataFrame({
        'Cisp_ID': cisp_value_counts.index,
        f'Count_of_{column_name}': cisp_value_counts.values
    })

    # Merge the original DataFrame with the count DataFrame based on 'Cisp'
    ndf = pd.merge(df, count_df, on='Cisp_ID', how='left')


    # Za (Equally distributed CISP): Each original 'column_name' value is divided by its count in the 'Cisp' column.
    # This step aims to distribute the values equally among the occurrences of each unique 'Cisp' value.
    ndf[f'Za_{column_name}'] = ndf[column_name] / ndf[f'Count_of_{column_name}']

    # Z (Weighted by non-CISP means): The equally distributed 'Cisp' values (Za) are then weighted by the corresponding non-CISP means.
    # This step aims to incorporate the non-CISP information in the redistribution process.
    ndf[f'Z_{column_name}'] = ndf[f'Za_{column_name}'] * ndf['noncisp_mean'] # non'column_name'


    # Group by 'Cisp' and sum for each cisp group
    # sum of Z/per cisp
    summed_Z = ndf.groupby('Cisp_ID')[f'Z_{column_name}'].sum().reset_index()

    # Merge to df
    ndf = pd.merge(ndf, summed_Z, on='Cisp_ID', how='left', suffixes=('', '_sum'))

    # Multiply Z (Nocisp Weighted by non-CISP means) / Z (original CISP means)
    # then divide by [sum of Z/per cisp]
    ndf[f'dist_{column_name}'] = (ndf[f'Z_{column_name}'] * ndf[column_name]) / ndf[f'Z_{column_name}_sum']

    # Fill NaN values with 0
    ndf.dropna(inplace=True)

    return ndf

In [20]:
dist_df = redistribute(crime_df,'cisp_mean')
dist_df[['Cisp_ID', 'cisp_mean', 'noncisp_mean', 'dist_cisp_mean']].head()

,Cisp_ID,cisp_mean,noncisp_mean,dist_cisp_mean
0,36.0,0.33599,0.563940,0.000088
1,36.0,0.33599,0.561908,0.000087
2,36.0,0.33599,0.562423,0.000087
3,36.0,0.33599,0.560358,0.000087
4,36.0,0.33599,0.558823,0.000087


Scale 'dist_cisp_mean' and 'Number_of_shootings' columns to the 0 to 1 range


In [15]:
# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the 'Number_of_shootings' column and transform it
grids['Number_of_shootings'] = scaler.fit_transform(grids[['Number_of_shootings']])
dist_df['dist_cisp_mean'] = scaler.fit_transform(dist_df[['dist_cisp_mean']])

Create crime index by combining 50% of 'shootings' and 50% of 'crime' values

In [16]:
newf = grids['Number_of_shootings'] * 0.5 + dist_df['dist_cisp_mean'] * 0.5

In [21]:

# Concatenate 'Redistributed CISP as safety_score' column to 'grid'
crime_index_grid = pd.concat([grids[['Id','geometry']], newf], axis=1)
crime_index_grid = gpd.GeoDataFrame(crime_index_grid, geometry='geometry')
crime_index_grid.rename(columns={0: 'crime_index'}, inplace=True)
crime_index_grid = crime_index_grid.dropna()

In [22]:
crime_index_grid.head() # crime index

,Id,geometry,crime_index
0,0,"MULTIPOLYGON (((623543.654 7466506.750, 623553...",0.000091
1,1,"MULTIPOLYGON (((623542.434 7466324.100, 623540...",0.000090
2,2,"MULTIPOLYGON (((623571.614 7466230.200, 623558...",0.000090
3,3,"MULTIPOLYGON (((623718.395 7466708.231, 623757...",0.000090
4,4,"MULTIPOLYGON (((623572.224 7466605.211, 623581...",0.000089


save to data folder

In [23]:
crime_index_grid_path = os.path.join(intermediate_output_folder_path, 'crime_index_grid.gpkg')


crime_index_grid.to_file(crime_index_grid_path, driver='GPKG')